In [86]:
#validators.py

def validate_string(value, name):
    value = str(value)
    if not len(value):
        raise ValueError(f'{name} must not be empty.')
    return value

def validate_integer(value, zero_allowed=False):
    try:
        value = int(value)
    except ValueError:
        raise TypeError('Value must be an integer.')
    if zero_allowed:
        if value < 0:
            raise ValueError('Value must not be negative.') 
    else:
        if value <= 0:
            raise ValueError('Value must be greater than zero.') 
    return value

def validate_storage_form_factor(value):
    if str(value).strip('"') not in ['2.5', '3.5']:
        raise ValueError('Form factor must be either 2.5" or 3.5".')
    else:
        return str(value).strip('"') + '"'

def validate_hdd_rpm(value):
    value = validate_integer(value)
    if not (5_000 <= value <= 15_000):
        raise ValueError('RPM must be between 5,000 and 15,000 RPM.')
    if value % 100:
        raise ValueError('RPM must be a multiple of 100.')
    return value



In [97]:
# inventory.py

# inventory.py

class Resource:

    def __init__(self, name, manufacturer, total=0, allocated=0):
        self._name = validate_string(name, 'Name')
        self._manufacturer = validate_string(manufacturer, 'Manufacturer')
        self._total = 0
        self._allocated = 0
        self.purchase(total, zero_allowed=True)
        self.claim(allocated, zero_allowed=True)

    @property
    def name(self):
        return self._name

    @property
    def manufacturer(self):
        return self._manufacturer

    @property
    def total(self):
        return self._total

    @property
    def category(self):
        return self.__class__.__name__.lower()

    def purchase(self, qty, zero_allowed=False):
        self._total += validate_integer(qty, zero_allowed=zero_allowed)
        self._remaining = None
        return f'New total quantity: {self.total}'

    @property
    def allocated(self):
        return self._allocated

    def claim(self, qty, zero_allowed=False):
        qty = validate_integer(qty, zero_allowed=zero_allowed)
        if qty > self.remaining:
            raise ValueError('Not enough available stock.')
        self._allocated += qty
        self._remaining = None
        return f'New allocated quantity: {self.allocated}'

    @property
    def remaining(self):
        if self._remaining is None:
            self._remaining = self.total - self.allocated
        return self._remaining

    def freeup(self, qty):
        qty = validate_integer(qty)
        self._allocated -= qty
        self._remaining = None
        return f'New allocated quantity: {self.allocated}'

    def died(self, qty):
        qty = validate_integer(qty)
        if qty > self.total:
            raise ValueError('Cannot kill more than total stock quantity.')
        else:
            self._total -= qty
            self._remaining = None
            if self.total <= self.allocated:
                self._allocated = self.total
        print(f'{qty} unit' + ('s' if qty > 1 else '') + ' died.'
               f'\nNew total: {self.total}'
               f'\nAllocated: {self.allocated}'
               f'\nRemaining: {self.remaining}')

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'{self.__class__.__name__} (name={self.name}, manufacturer={self.manufacturer})'

class CPU(Resource):
    def __init__(self, name, manufacturer, cores, socket, power_watts, total=0, allocated=0):
        self._cores = validate_integer(cores)
        self._socket = validate_string(socket, 'Socket')
        self._power_watts = validate_integer(power_watts)
        super().__init__(name, manufacturer, total=total, allocated=allocated)

    @property
    def cores(self):
        return self._cores

    @property
    def socket(self):
        return self._socket

    @property
    def power_watts(self):
        return self._power_watts

    def __repr__(self):
        return super().__repr__().rstrip(')') + (f', cores={self.cores}, '\
                                                 f'socket={self.socket}, '\
                                                 f'power={self.power_watts}W)')
    
class Storage(Resource):
    def __init__(self, name, manufacturer, capacity_GB, total=0, allocated=0):
        self._capacity_GB = validate_integer(capacity_GB)
        super().__init__(name, manufacturer, total=total, allocated=allocated)

    @property
    def capacity_GB(self):
        return self._capacity_GB

    def __repr__(self):
        return super().__repr__().rstrip(')') + (f', capacity={self.capacity_GB} GB)')
    



In [88]:
cpu = CPU('CPU', 'AMD', 8, 'AM4', 130, 100, 20)

In [89]:
cpu

CPU (name=CPU, manufacturer=AMD, cores=8, socket=AM4, power=130W)

In [90]:
cpu

CPU (name=CPU, manufacturer=AMD, cores=8, socket=AM4, power=130W)

In [91]:
validate_storage_form_factor(5.5-3.5)

ValueError: Form factor must be either 2.5" or 3.5".

In [92]:
validate_storage_form_factor(1)

ValueError: Form factor must be either 2.5" or 3.5".

In [93]:
validate_storage_form_factor('""""""""2.5"""""""""')

'2.5"'

In [94]:
validate_hdd_rpm('10_000')

10000

In [95]:
validate_hdd_rpm(10000)

10000

In [96]:
bool(100 % 1)

False

In [68]:
validate_hdd_rpm(10100)

10100

In [98]:
s = Storage('12TB Scorpion Black', 'Western Digital', 12_000, 30, 10)

In [99]:
s

Storage (name=12TB Scorpion Black, manufacturer=Western Digital, capacity=12000 GB)